In [1]:
#@title Download data from GCP bucket
import sys

if 'google.colab' in sys.modules:
  !gsutil -m cp -r gs://indaba-data .
else:
  !mkdir -p indaba-data/train
  !wget -P indaba-data/train https://storage.googleapis.com/indaba-data/train/train.csv --continue
  !wget -P indaba-data/train https://storage.googleapis.com/indaba-data/train/train_mut.pt --continue
  !wget -P indaba-data/train https://storage.googleapis.com/indaba-data/train/train_wt.pt --continue

  !mkdir -p indaba-data/test
  !wget -P indaba-data/test https://storage.googleapis.com/indaba-data/test/test.csv --continue
  !wget -P indaba-data/test https://storage.googleapis.com/indaba-data/test/test_mut.pt --continue
  !wget -P indaba-data/test https://storage.googleapis.com/indaba-data/test/test_wt.pt --continue

Copying gs://indaba-data/README.txt...
Copying gs://indaba-data/test/test.csv...
Copying gs://indaba-data/test/test_wt.pt...
Copying gs://indaba-data/test/test_mut.pt...
Copying gs://indaba-data/train/train.csv...
Copying gs://indaba-data/train/train_wt.pt...
Copying gs://indaba-data/train/train_mut.pt...
==> NOTE: You are downloading one or more large file(s), which would
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").



In [2]:
#@title Imports and moving to working directory
import torch 
import pandas as pd
from tqdm import tqdm
from torch.utils.data import DataLoader


# move to data folder
%cd indaba-data

/content/indaba-data


In [3]:
# Load Embedding tensors & Traing csv
# Embeddings were calculated using the ESM 650M pretrained model 
# Tensor shape of embedded data:  [data_len,1280] 
# There are no sequences in the Embedding tensors as we've performed an average of it (torch.mean(embed, dim=1))
# More details in https://huggingface.co/facebook/esm2_t33_650M_UR50D

wt_emb = torch.load("train/train_wt.pt")
mut_emb = torch.load("train/train_mut.pt")
df = pd.read_csv("train/train.csv")

In [ ]:
len(df)

339778

In [4]:
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

# Reset the index of the dataframe
df.reset_index(drop=True, inplace=True)

# Split data into train and validation
wt_emb_train, wt_emb_val, mut_emb_train, mut_emb_val, df_train, df_val = train_test_split(wt_emb, mut_emb, df, test_size=0.2, random_state=42)

# Define the dataset class
class EmbeddingDataset(Dataset):
  def __init__(self, wt_pt, mut_pt, data_df):
    self.pt_wt = wt_pt
    self.pt_mut = mut_pt
    self.df = data_df

  def __len__(self):
    return len(self.pt_wt)

  def __getitem__(self, index):
    if "ddg" in self.df.columns:
      df_out = torch.Tensor([self.df.iloc[index]["ddg"]])
    else:
      df_out = torch.Tensor([self.df.iloc[index]["ID"]])

    return self.pt_wt[index,:], self.pt_mut[index,:], df_out

# Create separate datasets for the training and validation sets
train_dataset = EmbeddingDataset(wt_emb_train, mut_emb_train, df_train.reset_index(drop=True))
val_dataset = EmbeddingDataset(wt_emb_val, mut_emb_val, df_val.reset_index(drop=True))

# Create dataloaders for the training and validation sets
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=2)


In [ ]:
len(train_dataloader)

10873

In [ ]:
len(val_dataloader)


2719

In [5]:
for i, (wt_emb, mut_emb, ddg) in enumerate(train_dataloader):
    print(f'wt_emb shape: {wt_emb.shape}, mut_emb shape: {mut_emb.shape}')
    if i > 5:  # Print shapes for first 5 batches only
        break
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

wt_emb shape: torch.Size([64, 1280]), mut_emb shape: torch.Size([64, 1280])
wt_emb shape: torch.Size([64, 1280]), mut_emb shape: torch.Size([64, 1280])
wt_emb shape: torch.Size([64, 1280]), mut_emb shape: torch.Size([64, 1280])
wt_emb shape: torch.Size([64, 1280]), mut_emb shape: torch.Size([64, 1280])
wt_emb shape: torch.Size([64, 1280]), mut_emb shape: torch.Size([64, 1280])
wt_emb shape: torch.Size([64, 1280]), mut_emb shape: torch.Size([64, 1280])
wt_emb shape: torch.Size([64, 1280]), mut_emb shape: torch.Size([64, 1280])


In [6]:
for wt_emb, mut_emb, ddg in train_dataloader:
    print(wt_emb_train.shape, mut_emb_train.shape, ddg.shape)
    break
import torch.nn as nn
import torch.nn.functional as F


torch.Size([271822, 1280]) torch.Size([271822, 1280]) torch.Size([64, 1])


In [7]:
import torch.nn as nn
import torch.nn.functional as F

class ProteinModel(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, num_layers):
        super(ProteinModel, self).__init__()
        
        self.conv1 = nn.Conv1d(embedding_dim, hidden_dim, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.conv2 = nn.Conv1d(hidden_dim, hidden_dim, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm1d(hidden_dim)
        self.lstm = nn.LSTM(hidden_dim, hidden_dim, num_layers=num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_dim*2, 1)
        
    def forward(self, wt_emb, mut_emb):
        x = wt_emb - mut_emb  # Use difference between embeddings as input
        x = x.unsqueeze(0)  # Add extra dimension for sequence length
        x, _ = self.lstm(x)
        x = self.fc(x[:, -1, :])
        return x


In [9]:
# Instantiate the model, loss function and optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ProteinModel(embedding_dim=1280, hidden_dim=512, num_layers=2).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

# Define a function for the training loop
def train(model, device, train_loader, criterion, optimizer, epoch):
    model.train()
    for batch_idx, (wt_emb, mut_emb, ddg) in enumerate(train_loader):
        wt_emb, mut_emb, ddg = wt_emb.to(device), mut_emb.to(device), ddg.to(device)
        optimizer.zero_grad()
        output = model(wt_emb, mut_emb)
        loss = criterion(output, ddg)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(ddg), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

# Define a function for the validation loop
def validate(model, device, val_loader, criterion):
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for wt_emb, mut_emb, ddg in val_loader:
            wt_emb, mut_emb, ddg = wt_emb.to(device), mut_emb.to(device), ddg.to(device)
            output = model(wt_emb, mut_emb)
            val_loss += criterion(output, ddg).item()  # sum up batch loss

    val_loss /= len(val_loader)
    print('\nValidation set: Average loss: {:.4f}\n'.format(val_loss))
    return val_loss

# Run the training and validation loops
num_epochs = 50
best_val_loss = float('inf')
for epoch in range(1, num_epochs + 1): 
    train(model, device, train_dataloader, criterion, optimizer, epoch)
    val_loss = validate(model, device, val_dataloader, criterion)
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model.pth')


IndexError: ignored

In [ ]:
input_dim = 1280
hidden_dim = 512
num_layers = 3
output_dim = 1

model = AdvancedBiLSTM(input_dim, hidden_dim, num_layers, output_dim)
model = model.to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    train_losses = []

    for wt_emb, mut_emb, ddg in train_dataloader:
        wt_emb, mut_emb, ddg = wt_emb.to(device), mut_emb.to(device), ddg.to(device)
        optimizer.zero_grad()
        outputs = model(wt_emb, mut_emb)
        loss = criterion(outputs, ddg)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())
    
    avg_train_loss = sum(train_losses) / len(train_losses)

    model.eval()
    with torch.no_grad():
        val_losses = []
        for wt_emb, mut_emb, ddg in val_dataloader:
            wt_emb, mut_emb, ddg = wt_emb.to(device), mut_emb.to(device), ddg.to(device)
            outputs = model(wt_emb, mut_emb)
            loss = criterion(outputs, ddg)
            val_losses.append(loss.item())
    
    avg_val_loss = sum(val_losses) / len(val_losses)

    print(f'Epoch {epoch+1}, Train Loss: {avg_train_loss}, Val Loss: {avg_val_loss}')

## Prediction & submission

In [ ]:
# load embedding tensors & traing csv
wt_test_emb = torch.load("test/test_wt.pt")
mut_test_emb = torch.load("test/test_mut.pt")
df_test = pd.read_csv("test/test.csv")

In [ ]:
# creating testing dataset and loading the embedding
test_dataset = EmbeddingDataset(wt_test_emb,mut_test_emb,df_test)
# preparing a dataloader for the testing
test_dataloader = torch.utils.data.dataloader.DataLoader(
        test_dataset,
        batch_size=32,
        shuffle=False,
        num_workers=2,
    )

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=model.to(device)
df_result = pd.DataFrame()
with torch.no_grad():
  for batch_idx, (data_mut,data_wt , target) in tqdm(enumerate(test_dataloader)):
    x1 = data_wt.to(device)
    x2 = data_mut.to(device)
    id = target.to(device)
    # make prediction
    y_pred = model(x1,x2)
    df_result = pd.concat([df_result, pd.DataFrame({"ID":id.squeeze().cpu().numpy().astype(int) , "ddg" : y_pred.squeeze().cpu().numpy()})])

In [ ]:
df_result.to_csv("submission_deepffnn.csv",index=False)